In [ ]:
# Importing useful libraries - YT: Multiple Regression Analysis in Python | Part 1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pylab
import math
import datetime

from scipy import stats
import statsmodels.api as sm
from statsmodels.stats import diagnostic as diag
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [ ]:
# Importing the database
df = pd.read_excel(r'C:\Users\danie\Documents\IMBA\SEMESTER 1\4. Research Methods & Data Analysis\3. Final report\Excel\FACTORS_x_Reg.xlsx', sheet_name = 'Kraken' )
dg = pd.read_excel(r'C:\Users\danie\Documents\IMBA\SEMESTER 1\4. Research Methods & Data Analysis\3. Final report\Excel\FACTORS_x_Reg.xlsx', sheet_name = 'Upbit' )
df = df.dropna(how = "any")
dg = dg.dropna(how = "any")

In [ ]:
# Set the index equal to the Date column
df.index = df['Date']
dg.index = dg['Date']

df = df.drop('Date', axis = 1)
dg = dg.drop('Date', axis = 1)

In [ ]:
# Set the data type of the data frame
df = df.astype(float)
dg = dg.astype(float)

# Check for nulls
display ("-"*100)
display(df.head())
display(dg.head())

#########################################################################################################################

In [ ]:
df = df.drop(['Mp(EW)', 'Mp(VW)', 'Size_1(EW)', 'Size_1(VW)', 'Size_2(EW)', 'Size_2(VW)', 'Size_3(EW)', 'Size_3(VW)', 'Mom_1(EW)', 'Mom_1(VW)', 'Mom_2(EW)', 'Mom_2(VW)', 'Mom_3(EW)', 'Mom_3(VW)', 'Illiq_1(EW)', 'Illiq_1(VW)', 'Illiq_2(EW)', 'Illiq_2(VW)', 'Illiq_3(EW)'], axis = 1)

# Define input variable and output variable
X = df.drop('Illiq_3(VW)', axis = 1)
Y = df[['Illiq_3(VW)']]

# Split dataset into training and testing portion
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = 1)

# Create an instance of our model
regression_model = LinearRegression()

# Fit the model
regression_model.fit(X_train, y_train)

In [ ]:
# PART 4
# Grab the intercept and the coefficients
intercept = regression_model.intercept_[0]
coef = regression_model.coef_[0]

print('The intercept of our model is {:.4}'.format(intercept))
print('-' * 100)

# Loop through the dictionary and print the data
for cf in zip(X.columns, regression_model.coef_[0]):
    print('The coefficient for {} is {:.2}'.format(cf[0], cf[1]))

In [ ]:
# Get multiple predictions
y_predict = regression_model.predict(X_test)

# Show the first five
y_predict[:5]

In [ ]:
# Evaluating the model
# Define our inputs

X2 = sm.add_constant(X)

# Create an OLS model
model = sm.OLS(Y, X2)

# Fit the data
est = model.fit()

In [ ]:
# Running White Test (Heteroscedasticity)
_, pval, _, f_pval = diag.het_white(est.resid, est.model.exog)
print(pval, f_pval)
print('-' * 100)

#print the results of the test
if pval > 0.05:
    print("For the White's test")
    print("The p-value was (:.4)".format(pval))
    print("We fail to reject the null hypothesis, so there is no heteroscedasticity. \n")
else:
    print("For the White's test")
    print("The p-value was (:.4)".format(pval))
    print("We reject the null hypothesis, so there is heteroscedasticity. \n")

In [ ]:
# Calculate the lag
lag = min(10, (len(X)// 5))
print('The number of lags will be ()'.format(lag))
print('-' * 100)

# Perform the Ljung-Box test
test_results = diag.acorr_ljungbox(est.resid, lags = lag)

print(test_results)
# Grab the p-values and test the statistics
ibvalue, p_val = test_results

# Print the results of the test
if min(p_val) > 0.05:
    print('The lowest p-value found was {:.4}'.format(min(p_val)))
    print('We fail to reject the null hypothesis, so there is no autocorrelation.')
    print('-' * 100)
else:
    print('The lowest p-value found was {:.4}'.format(min(p_val)))
    print('We reject the null hypothesis, so there is autocorrelation.')
    print('-' * 100)

# Plot autocorrelation
sm.graphics.tsa.plot_acf(est.resid)
plt.show()

In [ ]:
# PART 5

# Check for the normality of the residuals
sm.qqplot(est.resid, line = 's')
pylab.show()

# Check that the mean of the residuals is approximately 0
mean_residuals = sum(est.resid) / len(est.resid)
mean_residuals